In [80]:
import pandas as pd

In [81]:
movies = pd.read_csv('tmdb_5000_movies.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'tmdb_5000_movies.csv'

In [ ]:
movies.head(1)

In [ ]:
movies.shape

In [ ]:
#for i in movies.head(1)['genres']:
 #   print(i)

In [ ]:
#for i in movies['keywords']:
 #   print(i)

In [ ]:
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
credits.shape

In [ ]:
credits.head(1)

In [ ]:
data = movies.merge(credits, how='inner', left_on='id', right_on='movie_id')


In [ ]:
data

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
df = data[['id','genres','keywords','original_title','overview','tagline','cast', 'crew']]

In [ ]:
df

# Pre processing

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(inplace=True,axis=0)

In [ ]:
df.duplicated().sum()

In [ ]:
df.iloc[0].genres  

In [ ]:
import ast

In [ ]:
def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l    

In [ ]:
df['genres']=df['genres'].apply(convert)

In [ ]:
df

In [ ]:
df['keywords'] = df['keywords'].apply(convert)

In [ ]:
df.head(1)

In [ ]:
#df.iloc[0].cast

In [ ]:
def convert_cast(obj):
    l=[]
    counter = 0
    for i in ast.literal_eval(obj):
        if counter!=3:
            l.append(i['name'])
            counter+=1
        else:
            break
    return l    

In [ ]:
df['cast']=df.cast.apply(convert_cast)

In [ ]:
df.head(1)

In [ ]:
#df.iloc[0].crew

In [ ]:
def get_director(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            l.append(i['name'])
            break
    return l    

In [ ]:
df['crew'] = df.crew.apply(get_director)

In [ ]:
df.head(2)

In [ ]:
df.overview[0]

In [ ]:
df['overview'] = df.overview.apply(lambda x:x.split())

In [ ]:
df.overview[0]

In [ ]:
df.genres = df.genres.apply(lambda x:[i.replace(' ','') for i in x])
df.keywords = df.keywords.apply(lambda x:[i.replace(' ','') for i in x])
df.cast = df.cast.apply(lambda x:[i.replace(' ','') for i in x])
df.c = df.genres.apply(lambda x:[i.replace(' ','') for i in x])


In [ ]:
df.head(1)

In [ ]:
df['tags'] = df.overview+df.genres+df.keywords+df.cast+df.crew

In [ ]:
df.iloc[0].tags

In [ ]:
df.tags.apply(lambda x :''.join(x))
#for i in df.tags.apply(lambda x :' '.join(x)):
 #   print(i)
df.iloc[0].tags

In [ ]:
df.head(1)

In [ ]:
new_df = df[['id','original_title','tags']]

In [ ]:
new_df

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))
new_df

In [ ]:
new_df.head(5)

In [ ]:
#First  Concept Of Nlp One Hot Encoding. Basic encoding
color = ['red','blue','green','orange','red','blue','green']
df = pd.get_dummies(color, dtype=int)
print(df)

In [ ]:
# Sparcity : [0 0 0 0 0 1] here sparcity is high : A array have most zeros

In [ ]:
# 2nd concept : BWG : Binary bag of Words : For every 
# Vectorize The data in forrm of  list and make them Tokenize  
# Here We Check The Stirng : with every list values
# Bnifit Is We Have Less Sparcity.

In [ ]:
#Compress The List By removing The ( . , is , The etc) 
#Lemitizing  : Make The Multiple same words into (one - word) Give Un read word
#Stemking : Change The Word In Huyman redable word.

#Nlp : have two models : squence model ,  Bag Of Words model 

# Vectorization 

In [ ]:
# Bag of Words 
from sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer(max_features=5000, stop_words='english',binary=True)

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()
vectors.shape

In [ ]:
vectors[0].sum()

In [ ]:
len(cv.get_feature_names_out())

# Stemming

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stopwords.words('english')

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
new_df['tags'][0]

In [ ]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
stem("Students are leanring Python, A student ask about how to learn?")

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

# Similarity Check

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)


In [ ]:
similarity[0].shape

In [ ]:
# this will sort on new indices 
# sorted(list(enumerate(similarity[0])))


#this will held the indexes also 
sorted(list(enumerate(similarity[0])),reverse=True, key=lambda x: x[1])[1:10]

In [ ]:
new_df[new_df['original_title']=='Life of Pi']

In [ ]:
#Getting index of any movie
new_df[new_df['original_title']=='Life of Pi'].index[0]

In [ ]:
#getting title of movie using index
new_df.iloc[216]['original_title']

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['original_title']==movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)),reverse=True, key=lambda x: x[1])[1:6]
    for i in movie_list:
        print(new_df.iloc[i[0]]['original_title'])

In [152]:
recommend("The Dark Knight Rises")

NameError: name 'new_df' is not defined

In [ ]:
recommend("Pirates of the Caribbean: At World's End")

In [ ]:
res, scores = recommend('Independence Day')
#print(res)     # ['The Matrix', 'Guardians...', ...]
#print(scores)  # [0.78, 0.75, 0.72, ...]
import matplotlib.pyplot as plt

color = ['red','blue','green','yellow']
plt.bar(res, scores,color = color)   # ✅ bas lists use karo
plt.xticks(rotation=45)
plt.ylabel("Similarity Score")
plt.title("Top 5 Recommendations for Movies")
plt.show()


plt.plot(res, scores, marker='o',color='green',label='Movies')   
plt.xticks(rotation=45)
plt.ylabel("Similarity Score")
plt.title("Top 5 Recommendations for Movies")
plt.legend()
plt.show()


import random
import numpy as np
colors = ['red', 'green', 'blue', 'orange', 'purple']
size = np.random.randint(100, 120, len(res))
plt.scatter(res, scores, marker='o',label='Movies',s=size,c=colors)  
plt.xticks(rotation=45)
plt.ylabel("Similarity Score")
plt.title("Top 5 Recommendations for Movies")
plt.colorbar()
plt.legend()
plt.show()


In [154]:
d = pd.read_excel('Asad.xlsx')
d.head(2)

,EEID,Full Name,Job Title,Department,Business Unit,Gender,Ethnicity,Age,Hire Date,Annual Salary,Bonus %,Country,City,Exit Date
0,E02387,Emily Davis,Sr. Manger,IT,Research & Development,Female,Black,55,2016-04-08,141604,0.15,United States,Seattle,2021-10-16
1,E04105,Theodore Dinh,Technical Architect,IT,Manufacturing,Male,Asian,59,1997-11-29,99975,0.00,China,Chongqing,NaT


In [156]:
con = d.to_csv('Employe_data_Sales.csv')